In [1]:
import pandas as pd
from fractions import Fraction

csv_path = "Data/mr_boston_flattened.csv"

## Read CSV file

In [2]:
#read csv
cocktail_df = pd.read_csv(csv_path, encoding="utf-8")

## Define function ## 
to convert string values to number and clean measurement

In [3]:
#function for converting fractions and data cleanup
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        frac_str = frac_str.lower()
        if "or" in frac_str:
            nums = frac_str.split(' or ')
            return nums[0]
        elif "  " in frac_str:
            frac_str = frac_str.replace("  ", " ")
            print(frac_str)
            value = convert_to_float(frac_str)
            return value
        elif "tsp" in frac_str:
            frac_str = frac_str.strip("tsp")
            print(frac_str)
            value = convert_to_float(frac_str)
            return value * 0.166667
        elif "750-ml" in frac_str:
            frac_str = frac_str.strip("750-ml")
            print(frac_str)
            value = convert_to_float(frac_str)
            return value * 25.3
        elif "750ml" in frac_str:
            frac_str = frac_str.strip("750ml")
            print(frac_str)
            return 25.3
        elif "bottles" in frac_str:
            frac_str = frac_str.strip("bottles")
            print(frac_str)
            value = convert_to_float(frac_str)
            return value * 25.3
        elif "for glass" in frac_str:
            nums = frac_str.split('for glass')
            return 1
        elif "/" in frac_str:
            num, denom = frac_str.split('/')
            try:
                leading, num = num.split(' ')
                whole = float(leading)
            except ValueError:
                whole = 0
            frac = float(num) / float(denom)
            return whole - frac if whole < 0 else whole + frac    
        else:
            print("from else")
            print(frac_str)
            return frac_str

## Combine all Ingredient and Measure Columns

In [4]:
#create Dataframes for each ingredient subset
ing1_df = cocktail_df[["name", "measurement-1", "ingredient-1"]]
ing1 = ing1_df.rename(columns= {"measurement-1": "measurement", "ingredient-1":"ingredient"})
ing2_df = cocktail_df[["name", "measurement-2", "ingredient-2"]]
ing2 = ing2_df.rename(columns= {"measurement-2": "measurement", "ingredient-2":"ingredient"})
ing3_df = cocktail_df[["name", "measurement-3", "ingredient-3"]]
ing3 = ing3_df.rename(columns= {"measurement-3": "measurement", "ingredient-3":"ingredient"})
ing4_df = cocktail_df[["name", "measurement-4", "ingredient-4"]]
ing4 = ing4_df.rename(columns= {"measurement-4": "measurement", "ingredient-4":"ingredient"})
ing5_df = cocktail_df[["name", "measurement-5", "ingredient-5"]]
ing5 = ing5_df.rename(columns= {"measurement-5": "measurement", "ingredient-5":"ingredient"})
ing6_df = cocktail_df[["name", "measurement-6", "ingredient-6"]]
ing6 = ing6_df.rename(columns= {"measurement-6": "measurement", "ingredient-6":"ingredient"})

In [7]:
#combine data frames and sort
frames = [ing1, ing2, ing3, ing4, ing5, ing6]

working_df = pd.concat(frames)
working_df

,name,measurement,ingredient
0,Gauguin,2 oz,Light Rum
1,Fort Lauderdale,1 1/2 oz,Light Rum
2,Apple Pie,3 oz,Apple schnapps
3,Cuban Cocktail No. 1,1/2 oz,Juice of a Lime
4,Cool Carlos,1 1/2 oz,Dark rum
...,...,...,...
985,Wallis Blue Cocktail,NaN,NaN
986,Minnehaha Cocktail,NaN,NaN
987,Wallick Cocktail,NaN,NaN
988,Waikiki Beachcomber,NaN,NaN


In [39]:
working_df = new_cocktail_df.dropna(subset =['ingredient'])
reset_df = working_df.reset_index(inplace= False)
reset2_df = reset_df.drop('index', 1)
reset2_df['measurement'] = reset2_df['measurement'].fillna(1)
reset2_df

,name,measurement,ingredient
0,Gauguin,2 oz,Light Rum
1,Fort Lauderdale,1 1/2 oz,Light Rum
2,Apple Pie,3 oz,Apple schnapps
3,Cuban Cocktail No. 1,1/2 oz,Juice of a Lime
4,Cool Carlos,1 1/2 oz,Dark rum
...,...,...,...
3929,Bloody Maria,1,Lemon wheel
3930,Amante Picante,2 oz,"Jalapeno hot pepper sauce, cucumber slice"
3931,Betsy Ross Cocktail,1 1/4 oz,Old Mr. Boston Dry Gin
3932,The Winkle,4 oz,"Raspberries, lemon twist"


In [41]:
# #Add columns
# working_df['unit'] = ("")
# working_df['bool'] = ("")

In [50]:
reset2_df['bool'] = reset2_df['measurement'].str.contains(' oz')
reset2_df['bool'] = reset2_df['bool'].fillna('False')
reset2_df['unit'] = ""

In [54]:
for row, index in reset2_df.iterrows():
        if reset2_df['bool'][row] == True:
            reset2_df['unit'][row] = ("Fl Oz")
        else:
            reset2_df['unit'][row] = ("garnish")
            

In [55]:
reset2_df.to_csv("Data/ingredients.csv", index=False, header=True, encoding="UTF-8")

In [16]:
#strip oz
reset2_df['measurement']= liquid_df['measurement'].str.strip("oz")




NameError: name 'final_df' is not defined

In [17]:
ingred_df

NameError: name 'ingred_df' is not defined

## Run function and reindex

In [ ]:
ingred_df.to_csv("Data/ingredients.csv", index=False, header=True, encoding="UTF-8")

In [ ]:
#apply function to convert fractions
final_ingredient_df['measurement'] = final_ingredient_df['measurement'].apply(convert_to_float)

In [ ]:
# Loop through columns and strip units
for row, index in final_ingredient_df.iterrows():
        if cocktail_df['measurement'].str.contains(pat = 'oz'):
            cocktail_df['measurement'] = cocktail_df['measurement'].str.strip(" oz")
            coctail_df['unit']= "Fl Oz"
        else:
            coctail_df['unit']= ""
    
cocktail_df.head()

In [ ]:
for row, index in final_ingredient_df.iterrows():
    if isinstance(final_ingredient_df['measurement'][row], str):
        final_ingredient_df['unit'] = final_ingredient_df['measurement']
#         print('Type of variable is string')
    else:
        final_ingredient_df['unit'] = ('Fl Oz')
#         print('Type is variable is not string')

In [ ]:
# final_ingredient_df.ingredient.unique()

In [ ]:
final_ingredient_df.to_csv("Data/ingredients.csv", index=False, header=True, encoding="UTF-8")

In [ ]:
# # Create list of measurement columns to loop through
# ingredient_cols = ["ingredient-1","ingredient-2","ingredient-3","ingredient-4","ingredient-5","ingredient-6"]
# measurement_cols = ["measurement-1", "measurement-2", "measurement-3", "measurement-4", "measurement-5", "measurement-6"]

# # Loop through columns and strip units
# for col in measurement_cols:
#         cocktail_df[col]= cocktail_df[col].str.strip(" oz")
    
# cocktail_df.head()

In [ ]:
# from sqlalchemy import create_engine
# from config import postgres_pwd, postgres_user

# db_name = "cocktail_creator"
# table_name = "ingredients"

# connection_string = f"{postgres_user}:{postgres_pwd}@localhost:5432/{db_name}"
# engine = create_engine(f'postgresql://{connection_string}')
# conn = engine.connect()

# clean_df.to_sql(table_name, con=engine, if_exists="append", index=False)

# conn.close()